
![JupyterGames](./JupyterGames.png)


## Motivation

Making simple video games has always been a great way to learn programming for beginner.
While jupyter is the defacto standard for interactive programming in data science and machine learning,
they are not widely used for teaching programming and game development.


## The canvas

A drawable surface is the most basic requirement for any game development. In the web world, this is usually provided by the HTML5 `<canvas>` element.
In Jupyter, there are several ways to create and manipulate a canvas. The most popular one is the `ipycanvas` package, which provides a high-level API for creating and manipulating canvases in Jupyter notebooks.
However, `ipycanvas` is tied to the jupyter protocol, which makes it difficult to create a smooth gaming experience.

The jupyter protocol is a fire and forget protocol. Once a cell is executed, the kernel sends the output to the frontend and forgets about it.
This works well for data science and machine learning, where the output is usually a static image or text.
However, for game development, we need a way to continuously update the output at a fixed frame rate (e.g., 60 frames per second). 
While `ipycanvas` provides a way to update the canvas, it is impossible with ipycanvas to know when the frontend has actually rendered the updated canvas. This can lead to situations where the kernel is sending updates faster than the frontend can render them, leading to a choppy and unresponsive gaming experience.


## JuputerLite

[JupyterLite](https://jupyterlite.readthedocs.io/en/latest/) is a Jupyter distribution that runs entirely in the browser. It uses kernels compiled to WebAssembly like xeus-python, pyodide, xeus-r and even xeus-cpp, to run code directly in the browser without any server backend.
Jupyterlite supports widgets like `ipywidgets` and `ipycanvas` but is also tied to the jupyter protocol.

##

## The OffscreenCanvas 

The HTML5 OffscreenCanvas API provides a way to create a canvas that can be rendered offscreen, i.e., without being attached to the DOM. This allows us to create a canvas in the frontend / main-ui thread, and then transfer it to a web worker for rendering. 
Using the OffscreenCanvas API, we can create a custom canvas widget that creates an OffscreenCanvas in the frontend, transfers it to the kernel running in a web worker, and then uses it for rendering.
Since kernels like xeus-python provide a Javascript foreign function interface, we can directly call Javascript functions from Python , and use them to directely call the OffscreenCanvas API. Hence we can create a smooth gaming experience and render to the canvas at a fixed frame rate. That way, it is impossible to send updates faster than the frontend can render them, since the calls to the OffscreenCanvas API are blocking until the rendering is done.
That way we can bypass the limitations of the jupyter protocol and create a smooth gaming experience in JupyterLite.

We integrated the OffscreenCanvas API directely into the `ipycanvas` package and created a custom canvas widget called `OffscreenCanvas` which provides the same API as the regular `Canvas` widget from `ipycanvas`, but uses the OffscreenCanvas API for rendering.



## Overcome the protocol limitations for regular Jupyter

Since the OffscreenCanvas API is not available in regular Jupyter notebooks, we need to find a way to overcome the limitations of the jupyter protocol.
While its impossible with ipycanvas existing API to know when the canvas has been rendered in the frontned for the reason mentioned above, we can work around this limitation by creating a custom frontend extension that uses the following trick.

Asuming we want to know within a the exection of a single cell when some value of a widget has changed in the frontend,
we can utilize the following snippet that creates an asyncio.Future that will be set once the value of the widget changes in the frontend.

```python
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future
```

As an example, the following code creates an IntSlider widget and an async function that waits for the value of the slider to change.

```python
from ipywidgets import IntSlider
slider = IntSlider()

async def f():
    for i in range(5):
        print('did work %s'%i)
        x = await wait_for_change(slider, 'value')
        print('async function continued with value %s'%x)
asyncio.ensure_future(f())

slider
```

In [ ]:
import asyncio
from ipywidgets import IntSlider

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

slider = IntSlider()

async def f():
    for i in range(5):
        print('did work %s'%i)
        x = await wait_for_change(slider, 'value')
        print('async function continued with value %s'%x)
asyncio.ensure_future(f())

slider

Using the above trick, we can create a custom canvas widget that will notify the kernel once the canvas has been rendered in the frontend. This way, we can create a game loop that runs at a fixed frame rate and only updates the canvas once the previous frame has been rendered.

# Box2d

[Box2D](https://box2d.org/) is a popular 2D physics engine that is widely used in game development. It provides a simple and efficient way to simulate physics in 2D games.
Recently, there has been a major update to the Box2D codebase, namely Box2D 3.0 which introduces several new features and dramatic performance improvements over the previous versions.

## Python wrapper for Box2D 3.0
We have created a Python wrapper for Box2D 3.0 called [pyb2d3](https://github.com/DerThorsten/pyb2d3/) that allows you to use Box2D 3.0 in Python. The wrapper is built using [nanobind](https://nanobind.github.io/) and provides a simple and efficient way to use Box2D 3 in Python.
To be able to use pyb2d3 in jupyterlite  we created a wasm build of pyb2d3 using [emscripten](https://emscripten.org/) and the  [emscripten-forge](https://emscripten-forge.org/) conda [distribution](https://prefix.dev/channels/emscripten-forge-dev).

## Box2D sandbox

Box2d and its python wrapper pyb2d3 do not provide any rendering capabilities out of the box. However both Box2D and pyb2d3 provide a "sandbox" playground that allows you to quickly create simple Box2D which are automatically rendered.

For pyb2d3, we have created a custom package for this sandbox called `pyb2d3-sandbox` with various frontends, including a jupyter notebook frontend that uses the custom canvas widget described above to provide a smooth gaming experience in jupyter notebooks.

These frontends are:

 * pyb2d3-sandbox-opengl: A desktop OpenGL frontend for pyb2d3-sandbox using PyOpenGL and dearimgui
 * pyb2d3-sandbox-pygame: A desktop pygame frontend for pyb2d3-sandbox using pygame for rendering
 * pyb2d3-sandbox-ipycanvas: A Jupyter notebook frontend for pyb2d3-sandbox using the custom OffscreenCanvas widget when in JupyterLite
 * pyb2d3-sandbox-jupyter: A Jupyter notebook frontend for pyb2d3-sandbox using `wait_for_change` trick when in regular Jupyter notebooks


In [ ]:
import pyb2d3 as b2d
from pyb2d3_sandbox import SampleBase


class NewtonsCradle(SampleBase):
    def __init__(self, frontend, settings):
        super().__init__(frontend, settings)

        # physical world
        self.n_balls = 10
        self.radius = 1
        self.actual_radius = self.radius * 0.85
        self.rope_length = 10

        diameter = self.radius * 2
        for i in range(self.n_balls):
            x = diameter * i
            y_ball = 0
            y_rope = self.rope_length

            # create dynamic body for the ball
            ball_body = self.world.create_dynamic_body(
                position=(x, y_ball), linear_damping=0.1, angular_damping=0.0
            )
            ball_body.awake = True
            # create circle shape for the ball
            material = b2d.surface_material(
                restitution=1.0,
                friction=0.0,
                custom_color=b2d.hex_color(100, 0, 200),
            )
            ball_body.create_shape(
                b2d.shape_def(density=1, material=material),
                b2d.circle(radius=self.actual_radius),
            )

            # create a rope anchor for the balls
            anchor_pos = (x, y_rope)
            anchor_body_id = self.world.create_static_body(position=anchor_pos)

            self.world.create_distance_joint(
                body_a=ball_body,
                body_b=anchor_body_id,
                length=self.rope_length,
                enable_spring=False,
            )

        impulse = (-10, 0)
        ball_body.apply_linear_impulse_to_center(impulse, wake=True)

    def aabb(self):
        return b2d.aabb(
            lower_bound=(-(self.rope_length + 2 * self.radius), 0),
            upper_bound=(
                self.n_balls * self.radius * 2 + self.rope_length,
                self.rope_length + 2 * self.radius,
            ),
        )

    def on_key_down(self, event):
        if event.key == "a":
            # create dynamic body for the ball
            ball_body = self.world.create_dynamic_body(
                position=(0, 0), linear_damping=0.1, angular_damping=0.0
            )
            ball_body.awake = True
            # create circle shape for the ball
            material = b2d.surface_material(
                restitution=1.0,
                friction=0.0,
                custom_color=b2d.hex_color(100, 0, 200),
            )
            ball_body.create_shape(
                b2d.shape_def(density=1, material=material),
                b2d.circle(radius=self.actual_radius),
            )

if __name__  == "__main__":
    NewtonsCradle.run(frontend_settings=dict(simple_ui=True, autostart=True))